In [ ]:
import json
import requests
import urllib3
from datetime import datetime, timedelta

In [ ]:
config_file = "config.json"

In [ ]:
# deshabilitar la advertencia de InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# Funcion para leer configuracion desde JSON
def load_config(config_file):
    with open(config_file, "r") as file:
        return json.load(file)

In [ ]:
# Función para calcular fechas de filtro
def calculate_date_filters():
    now = datetime.utcnow()
    twenty_four_hours_ago = now - timedelta(hours=24)
    return {
        "now": now.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "24_hours_ago": twenty_four_hours_ago.strftime("%Y-%m-%dT%H:%M:%SZ")
    }

In [ ]:
# Llamada genérica a la API
def fetch_data(endpoint, headers=None):
    try:
        response = requests.get(endpoint, headers=headers, verify=False)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error al obtener datos de la API: {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Error en la solicitud de la API: {e}")
        return None

In [ ]:
def filter_fields(data, fields):
    # Si `data` es una lista de entradas, se aplica a cada entrada
    if isinstance(data, list):
        return [{key: item[key] for key in fields if key in item} for item in data]
    # Si `data` es un solo diccionario, aplica directamente
    elif isinstance(data, dict):
        return {key: data[key] for key in fields if key in data}
    return data

In [ ]:
# Guardar los datos en un archivo JSON
def save_json(data, system, instance, query_name):
    output_file = f"{system}-{instance}-{query_name}.json"
    with open(output_file, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Datos guardados en: {output_file}")

In [ ]:
# Función para obtener el token de autenticación de PPDM
def get_token_PPDM (ppdm_instance):
    url = f'https://{ppdm_instance}:8443/api/v2/login'
    headers = {
        'Content-Type': 'application/json'
    }

    # Credenciales desde variables de entorno
    data = {
        "username" : 'DCOapi',
        "password" : 'Password123!'
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)
        if response.status_code == 200:
            response_json = response.json()
            access_token = response_json.get('access_token')
            refresh_token = response_json.get('refresh_token')
            return access_token, refresh_token
        else:
            print(f"Error al obtener el token: {response.status_code}")
            return None, None
    except requests.RequestException as e:
        print(f"Error en la solicitud de autenticación: {e}")
        return None, None

In [ ]:
def fetch_system_health_issues(instance, access_token):
    headers = {"Authorization": f"Bearer {access_token}"}  # Añade "Bearer " si es necesario
    fields = [
        "healthCategory", "severity", "scoreDeduction", "componentType",
        "componentName", "messageID", "detailedDescription", "responseAction"
    ]
    endpoint = f"https://{instance}:8443/api/v2/system-health-issues"
    print(endpoint)

    # Obtén los datos sin filtrado inicial
    data = fetch_data(endpoint, headers)

    # Aplica el filtrado si los datos se recibieron correctamente
    if data:
        filtered_data = filter_fields(data, fields)
        return filtered_data
    return None


In [ ]:
def main():
    config = load_config(config_file)
    
    for system, instances in config["systems"].items():
        for instance in instances:
            # Obtener token de autenticación
            access_token, _ = get_token_PPDM(instance)
            print(system, instance, access_token)
            if not access_token:
                print(f"Error: no se pudo obtener el token para {instance}.")
                continue

            if system == "PPDM":
                data = fetch_system_health_issues(instance, access_token)
               
                if data:
                    save_json(data, system, instance, "system_health_issues")
    

In [ ]:
if __name__ == "__main__":
    main()